In [1]:
import pandas as pd
import numpy as np
import csv
import os
from os import path
import weblogo
import seqlogo
%matplotlib inline

In [2]:
filteredGeneList = pd.read_csv('../../Database/filteredGenesDetails_human_240118.txt',sep='\t')
acc2gene={}
gene2acc={}
for idx,row in filteredGeneList.iterrows():
    acc2gene[row['AccNum']] = row['GeneName']
    gene2acc[row['GeneName']]=row['AccNum']

In [3]:
def get_fa(filename,onlyKeys = 'All'):
    keyD = dict()
    if onlyKeys!='All':
        for k in onlyKeys: keyD[k] = None
        
    def addEntry(current,d):
        entry = current.split('\n')
        if entry[0]!='' and (onlyKeys=='All' or keyD.has_key(entry[0])):
            d[entry[0]] = ''.join(entry[1:])

    d = dict()
    bf = open(filename)
    current = ['\n']
    for line in bf:
        if line[0]=='>':
            addEntry(''.join(current),d)
            if len(line)>1: current = [line[1:]]
            else: current = ['\n']
        else: current.append(line)
    addEntry(''.join(current),d)
    bf.close()
    return d

In [4]:
splicedGenes = get_fa('../../Database/splicedGenes_240118.txt')
## truncate the keys to only accNum
ks = list(splicedGenes.keys())
for key in ks:
    splicedGenes[key.split('_chr')[0]] = splicedGenes[key]
    splicedGenes.pop(key)
splicedORFs = get_fa('../../Database/splicedORFs_240118.txt')

In [5]:
fpUTR={}
tpUTR = {}
ORFs = {}
for idx,row in filteredGeneList.iterrows():
    accNum = row['AccNum']
    fpLen = row['fpUTR_length']
    tpLen = row['tpUTR_length']
    fpUTR[accNum] = splicedGenes[accNum][0:fpLen]
    tpUTR[accNum] = splicedGenes[accNum][-tpLen:]
    ORFs[accNum] = splicedORFs[accNum]

In [6]:
class Sequences:
    nt2coord = {'A':0,'C':1,'G':2,'T':3}
    
    def seqLikelihood(self,SEQ):
        nt2coord = {'A':0,'C':1,'G':2,'T':3}
        prob = 1
        assert (len(SEQ)==19)
        for i in range(8):
            pos_prob = self.ppm[i,nt2coord[SEQ[i]]]
            prob*= pos_prob
        for i in range(11,19):
            pos_prob = self.ppm[i,nt2coord[SEQ[i]]]
            prob*= pos_prob
        return np.log(prob)
    
    @staticmethod
    def genes2seq(genes):
        seqs = []
        for gene in genes:
            fp=fpUTR[gene]
            if len(fp)<8:
                #print('No fpUTR encountered: ',acc2gene[gene])
                continue
            seq = fp[-8:]+ORFs[gene][:11]
            seqs.append(seq)
        #print('# of valid seqs: ',len(seqs))
        return seqs
    
    def __init__(self,seqs):
        self.seqs=seqs
        self.pfm, self.ppm = self.comp_count()
        self.flat = self.ppm2flat()
    
    def addseqs(self,seqs):
        self.seqs.extend(seqs)
        self.pfm,self.ppm = self.comp_count()
        self.flat = self.ppm2flat()
    
    def __len__(self):
        return len(self.seqs)
    
    def comp_count(self,seqLen=19):
        
        raw_count = np.zeros((seqLen,4))
        lines_tr=[''.join(s) for s in zip(*(self.seqs))]
        #print('#lines = ',len(seqs))
        for i in range(seqLen):
            raw_count[i][0] = lines_tr[i].count('A')
            raw_count[i][1] = lines_tr[i].count('C')
            raw_count[i][2] = lines_tr[i].count('G')
            raw_count[i][3] = lines_tr[i].count('T')
        ppm = self.pfm2ppm(raw_count)
        return raw_count, ppm
    
    ## Note: ppm here is the same as the so-called pwm in R,
    ## True meaning of 'pwm' vary from literatures
    def pfm2ppm(self,pfm):
        sum_of_rows = pfm.sum(axis=1)
        return pfm / sum_of_rows[:, np.newaxis]
    
    def plotlogo(self):
        pfm_pd=pd.DataFrame(self.pfm)
        ppm_pd=seqlogo.pfm2ppm(pfm_pd)
        ppm = seqlogo.Ppm(ppm_pd)
        plt=seqlogo.seqlogo(ppm, ic_scale = True, format = 'png', size = 'large')#,filename=datset+'.png')
        #seqlogo.seqlogo(ppm, ic_scale = True, format = 'svg', size = 'medium')#,filename=datset+'.svg')
        return plt
    
    def ppm2flat(self):
        return self.ppm[list(range(0,8))+list(range(11,19)), 0:4].reshape(-1)
    
    
    def bootstrap(self,num):    # with replacement
        sample_seqs = np.random.choice(self.seqs, size=num)
        return Sequences(sample_seqs.tolist())
    
    def sample(self,num): # without replacement
        sample_seqs = np.random.choice(self.seqs, size=num,replace=False)
        return Sequences(sample_seqs.tolist())

    def __gt__(self, seq2):
        return True

In [31]:
from scipy.stats import chisquare
import seaborn as sns
class TestSeq:
    
    def __init__(self,target,test,resample=10000):
        self.resample=resample
        self.target=target
        self.test=test
        self.ps, self.seqs=self.createTest(target,test,resample)
        
    @classmethod
    def createTest(cls,target,test,resample=10000):
        ps = []
        seqs=[]
        for i in range(0,resample):
            seq = test.bootstrap(len(target))
            ## dof = (4-1)*(8+8) = 48, number of freqs = 64
            ## dof = k - 1 - ddof
            ## ddof = 64 - 1 - 48 = 15
            _, p = chisquare(seq.flat*len(target),f_exp=target.flat*len(target),ddof=14)
            ps.append(p)
            seqs.append(seq)
        both = sorted(zip(ps,seqs))
        ps,seqs = [y for y,x in both],[x for y,x in both]
        return ps, seqs

    def dist(self):
        return sns.distplot(self.ps)
    def logdist(self):
        return sns.distplot(np.log2(self.ps))

In [8]:
def pfm2ppm_df(pfm):
    pfm_pd=pd.DataFrame(pfm)
    ppm_pd=seqlogo.pfm2ppm(pfm_pd)
    return seqlogo.Ppm(ppm_pd).T

In [16]:
batch2seq={}

In [24]:
batch = 'ER_S15_t2'
deseq = pd.read_csv('../../Deseq2 Analysis/shift15/coding/csv_raw/'+batch+'.csv')
deseq = deseq.sort_values(by=['log2FoldChange'])
print(len(deseq))
deseq.head(2)

7990


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
7154,NM_001195144,274.30340,-1.464396,0.417503,-3.507511,0.000452,0.040863
5547,NM_001324355,50.93417,-1.411277,0.467037,-3.021764,0.002513,0.127085


In [25]:
background = deseq['Unnamed: 0']
deseq_filtered = deseq.loc[deseq['padj']<0.05]
up_num = sum(deseq_filtered['log2FoldChange'] > 1)
target = deseq_filtered.tail(up_num)['Unnamed: 0']   # i.e. Interest group, upper right on volcano plot
down_num=sum(deseq_filtered['log2FoldChange']<-1)
print(up_num,down_num)

33 1


In [26]:
target_seq = Sequences(Sequences.genes2seq(target))
bg_seq = Sequences(Sequences.genes2seq(background))
batch2seq[batch] = target_seq
batch2seq['background'] = bg_seq

In [12]:
import glob
glob.glob('kmeans_jm/*')

['kmeans_jm\\cluster 1 genes.txt',
 'kmeans_jm\\cluster 2 genes.txt',
 'kmeans_jm\\cluster 3 genes.txt',
 'kmeans_jm\\cluster 4 genes.txt',
 'kmeans_jm\\cluster 5 genes.txt',
 'kmeans_jm\\cluster 6 genes.txt',
 'kmeans_jm\\cluster 7 genes.txt',
 'kmeans_jm\\cluster 8 genes.txt',
 'kmeans_jm\\logos']

In [32]:
for i in range(1,9):
    newpath = 'kmeans_jm/logos' 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    
    f = pd.read_csv('kmeans_jm\\cluster '+str(i)+' genes.txt',header=None)
    genes = list(f[0])
    cluster_seq = Sequences(Sequences.genes2seq(genes))
    img=cluster_seq.plotlogo()
    #display(img)
    with open("kmeans_jm/logos/cluster "+str(i)+" logo.png", "wb") as png:
        png.write(img.data)
    ppm_out = pfm2ppm_df(cluster_seq.pfm)
    ppm_out.to_csv("kmeans_jm/logos/cluster"+str(i)+"_pwm.csv")

In [43]:
import matplotlib.pyplot as plt
for i in range(1,9):
    f = pd.read_csv('kmeans_jm/genes/cluster '+str(i)+' genes.txt',header=None)
    genes = list(f[0])
    cluster_seq = Sequences(Sequences.genes2seq(genes))
    np.random.seed(42)
    for batch in ['ER_S15_t0','ER_S15_t1','ER_S15_t2','ER_L24_t1']:
        test = TestSeq(batch2seq[batch],cluster_seq,100000)
        with open('kmeans_jm/'+batch+"/cluster"+str(i)+"_test_ps.txt", 'w') as f:
            for item in test.ps:
                f.write("%s\n" % item)
        plt.figure()
        ax=test.logdist()
        ax.get_figure().savefig('kmeans_jm/'+batch+"/cluster"+str(i)+"_test_hist.png")
        plt.close()